In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Natural Language Processing (NLP)

In [60]:
PS3train = pd.read_csv("PS3_train.csv")
PS3test = pd.read_csv("PS3_test.csv")

In [64]:
PS3train

,content,title,uid,target_ind
0,Premium quality five pocket jean from Wrangler...,Amazon.com: Wrangler Men's Rugged Wear Relaxed...,B0000CBALT,247
1,If you're looking for a different kind of anim...,Sakura Diaries - Complete Series Collector's E...,B00005QFDT,453
2,"First things first: Yes, Thinking XXX features...",Thinking XXX (Extended Cut) (2006),B000BNXD50,228
3,Feathertouch. 100% Polyester Machine Wash Warm...,Amazon.com: Petite Feathertouch Pull-On Pant: ...,B0002LK9V2,223
4,"When you need outstanding fuel delivery, easy ...",ACDelco EP386 Fuel Pump,B000C9PA54,312
...,...,...,...,...
35107,"SArah Walker, Anthony Rolfe Johnson, Jean Rigb...",Britten - Gloriana (1984),B000I2IUM0,473
35108,Buy Teenage Mutant Ninja Turtles 4: Turtles in...,Teenage Mutant Ninja Turtles IV: Turtles in Time,B000035Y3N,74
35109,A 10 movie collection of women action flicks. ...,Women Who Kick Butt 10 Movie Pack (2001),B00005B7BW,460
35110,This huarache sandal is designed for all-day c...,Softspots Women's Tela Leather Sandal,B0002KIFAK,158


In [67]:
PS3train = PS3train.reindex(columns = ["target_ind", "content", "title", "uid"])

In [68]:
PS3train

,target_ind,content,title,uid
0,247,Premium quality five pocket jean from Wrangler...,Amazon.com: Wrangler Men's Rugged Wear Relaxed...,B0000CBALT
1,453,If you're looking for a different kind of anim...,Sakura Diaries - Complete Series Collector's E...,B00005QFDT
2,228,"First things first: Yes, Thinking XXX features...",Thinking XXX (Extended Cut) (2006),B000BNXD50
3,223,Feathertouch. 100% Polyester Machine Wash Warm...,Amazon.com: Petite Feathertouch Pull-On Pant: ...,B0002LK9V2
4,312,"When you need outstanding fuel delivery, easy ...",ACDelco EP386 Fuel Pump,B000C9PA54
...,...,...,...,...
35107,473,"SArah Walker, Anthony Rolfe Johnson, Jean Rigb...",Britten - Gloriana (1984),B000I2IUM0
35108,74,Buy Teenage Mutant Ninja Turtles 4: Turtles in...,Teenage Mutant Ninja Turtles IV: Turtles in Time,B000035Y3N
35109,460,A 10 movie collection of women action flicks. ...,Women Who Kick Butt 10 Movie Pack (2001),B00005B7BW
35110,158,This huarache sandal is designed for all-day c...,Softspots Women's Tela Leather Sandal,B0002KIFAK


# Torchtext for classifying text

In [69]:
import torchtext

 Basic data processing building blocks for raw text string.
* `tokenizer` - for separating the english basics into tokens
* `vocabulary` - The vocabulary block converts a list of tokens into integers.

In [70]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [104]:
tokenizer = get_tokenizer("basic_english")

In [105]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [106]:
yield_tokens(iter(PS3train))

<generator object yield_tokens at 0x0000021271E34200>

In [112]:
vocab = build_vocab_from_iterator(yield_tokens(iter(PS3train).items()), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

AttributeError: 'map' object has no attribute 'items'

In [75]:
vocab

Vocab()

In [76]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x : int(float(x)) - 1

In [77]:
text_pipeline('here is the an example')

[465, 9, 3, 28, 1667]

In [78]:
text_pipeline(PS3train.iloc[0, 1])


[683,
 243,
 684,
 735,
 1351,
 26,
 1909,
 1347,
 215,
 2,
 16,
 1471,
 202,
 1351,
 9,
 125,
 26,
 526,
 232,
 2073,
 10,
 424,
 13,
 521,
 1335,
 8,
 3,
 1789,
 4,
 3116,
 10,
 219,
 2,
 523,
 14,
 17,
 1909,
 1698,
 4464,
 1471,
 202,
 2572,
 197,
 117,
 18,
 5,
 275,
 8638,
 1,
 25,
 3297,
 209,
 10,
 28,
 6221,
 307,
 6,
 2460,
 1544,
 2471,
 26,
 3,
 219,
 6,
 32,
 304,
 487,
 2,
 70,
 1909,
 1698,
 4464,
 1471,
 202,
 2572,
 15,
 87,
 10,
 1440,
 23,
 15,
 62,
 1810,
 14,
 3269,
 117,
 526,
 232,
 2073,
 321,
 1471,
 184,
 735,
 233,
 185,
 3147,
 1,
 521,
 979,
 519,
 826,
 738,
 392,
 9252,
 5373,
 2122,
 210,
 1107,
 2415,
 202,
 315,
 105,
 1444,
 261,
 969,
 860,
 722,
 2,
 328,
 2,
 857,
 89,
 4,
 76,
 62,
 258,
 2963,
 557,
 62,
 523,
 14,
 17,
 1909,
 2944,
 1487,
 1698,
 4464,
 1471,
 202,
 2572]

In [79]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [214]:
batch = PS3train.iloc[1:, :2]

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch.items():
        label_list = torch.tensor(label_pipeline(_label), dtype = torch.int64)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
        label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader((iter(PS3train.iloc[:].items())), batch_size=8, shuffle=False, collate_fn=collate_batch)

In [215]:
batch

,target_ind,content
1,453,If you're looking for a different kind of anim...
2,228,"First things first: Yes, Thinking XXX features..."
3,223,Feathertouch. 100% Polyester Machine Wash Warm...
4,312,"When you need outstanding fuel delivery, easy ..."
5,85,Cart only.
...,...,...
35107,473,"SArah Walker, Anthony Rolfe Johnson, Jean Rigb..."
35108,74,Buy Teenage Mutant Ninja Turtles 4: Turtles in...
35109,460,A 10 movie collection of women action flicks. ...
35110,158,This huarache sandal is designed for all-day c...


In [82]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [95]:
num_class = len(set([label for (label, text) in iter(PS3train.iloc[:, 0:2].items())]))
num_class

2

In [96]:
vocab_size = len(vocab)
vocab_size

99076

In [85]:
num_class = int(torch.max(torch.tensor(np.array(PS3train.iloc[:, 3], dtype = np.float32))))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

ValueError: could not convert string to float: 'B0000CBALT'

In [86]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0


In [87]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [88]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_dataset = to_map_style_dataset(PS3train.items())
test_dataset = to_map_style_dataset(PS3test.items())
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

AttributeError: 'list' object has no attribute 'items'